In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from general_functions.datetime_helper import transform_date_to_timestamp_milliseconds
from general_functions.return_account_ids import return_account_ids, return_accounts
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [10]:
accounts = return_accounts()
from_date = "20250826"
to_date ="20250827"
len(accounts)
url = return_api_url()
print(f"url = {url}")

In [22]:
res = pd.DataFrame(columns=["account", "revenue"])
for account in accounts:
    revenue = False
    account_id = account["id"]
    account_name = account["name"]
    print(account_name)
    if account_name in ["Inside Beauty Magazine"]:
        continue
    content={
     "created": {
                     "$gte": transform_date_to_timestamp_milliseconds(from_date),
                     "$lte": transform_date_to_timestamp_milliseconds(to_date),
               }
                }
    conversions=send_to_innkeepr_api_paginated(
        f"{url}/conversions/query",
        account_id,
        content,
        logging
    )
    conversions = pd.json_normalize(conversions)
    if "properties.revenue" in conversions:
        temp = conversions["properties.revenue"].astype("float").sum()
        if temp > 0:
            revenue = True
    res.loc[len(res)] = [account_name, revenue]
res

In [28]:
res_cleaned = res[res["account"].isin(["Paul Valentine","Sospree","NoCoffee","Beyond Love"])==False]
res_cleaned


In [29]:
res_cleaned["revenue"].sum()/len(res_cleaned)

In [27]:
res_cleaned[res_cleaned["revenue"]==False]["account"].tolist()